# Category browser for the SA Government Photographic Collection

In [105]:
import requests
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, Image, HTML
from io import BytesIO

In [3]:
API_URL = 'http://data.history.sa.gov.au/sagpc'

In [146]:
def format_category_name(category):
    # There are a couple of top-level categories that are just containers, so we'll ignore them
    level = category['level'] - 2
    if level:
        indent = f' {level * "-"} '
    else:
        indent = ''
    return f'{indent}{category["name"]}'

def make_category_dropdown_data():
    '''
    Generate a list of label/id paris for use in a dropdown selector
    '''
    # Load the category data
    df = pd.read_csv('sa-gov-photo-categories.csv')
    # Create labels from names with identation by level (to show hierarchy)
    df['labels'] = df.apply(format_category_name, axis='columns')
    # Return labels/ids as a list of two element lists
    options = df[['labels', 'id']].loc[df['level'] > 1].to_dict('split')['data']
    return options

def get_results(change):
    '''
    When a category is selected, get results for the category.
    Display image and details for each item.
    '''
    if change['type'] == 'change' and change['name'] == 'value':
        out.clear_output()
        # Search for category
        response = requests.get(f'{API_URL}/search/category/{change.new}')
        data = response.json()
        with out:
            display(HTML(f'{data["totalcount"]} results'))
            # Display items
            for item in data['items']:
                # print(item)
                image = widgets.HTML(f'<a href="{API_URL}/media/preview/{item["id"]}"><img src="{API_URL}/media/preview/{item["id"]}?maxsize=400" style="margin-top:20px; margin-right: 20px;"></a>')
                caption = widgets.HTML(f'<h4>{item["Title"]}</h4><p>{item["Description"]}</p>')
                display(widgets.HBox([image, caption]))
                                                  

In [145]:
# Create dropdown selector
categories = widgets.Dropdown(options=make_category_dropdown_data())
# When the selection changes do a search
categories.observe(get_results)
out = widgets.Output()
display(categories)
display(out)

Dropdown(options=(['Clothing and Dress', 302], [' - Clothing 1900-1910', 354], [' - Clothing 1910-1920', 375],…

Output()